In [1]:
# !chmod 400 /Users/jackhan/Desktop/Alpfin/OneZero_Data.pem
# !ssh-add -d ~/.ssh/id_ed25519

In [2]:
import requests

# Fetch public IP
response = requests.get("http://checkip.amazonaws.com")
if response.status_code == 200:
    public_ip = response.text.strip()
    print(f"Public IP: {public_ip}")
else:
    print(f"Failed to fetch public IP. Status code: {response.status_code}")

# The public IP should be in the whitelist of the AWS server, contact Sabah Hussain for access 

Public IP: 27.18.230.250


In [3]:
import pymysql
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import os
import time

def get_quote_data(date, symbol):
    """
    Fetch quote data for a specific date and symbol from Alp_Quotes.
    Checks if data already exists in a file; skips query if it does.
    """
    
    # -------------------------------
    # 1. Build the partition name
    # -------------------------------
    month_map = {
        1: "jan", 2: "feb", 3: "mar", 4: "apr", 5: "may", 6: "jun",
        7: "jul", 8: "aug", 9: "sep", 10: "oct", 11: "nov", 12: "dec"
    }
    partition_name = f"p_{month_map[date.month]}_{date.year}"
    
    # -------------------------------
    # 2. Build time filter boundaries
    # -------------------------------
    if not isinstance(date, pd.Timestamp):
        date = pd.Timestamp(date)
    start_str = date.strftime("%Y-%m-%d 00:00:00")
    end_str = (date + pd.Timedelta(days=1)).strftime("%Y-%m-%d 00:00:00")
    
    # -------------------------------
    # 3. Clean up symbol for file naming
    # -------------------------------
    symbol_filename = symbol.replace('/', '')
    date_str = date.strftime("%Y-%m-%d")
    output_pickle = f"Data/{symbol_filename}_{date_str}.pkl"
    
    # -------------------------------
    # 4. Check if file already exists
    # -------------------------------
    if os.path.exists(output_pickle):
        print(f"File {output_pickle} already exists. Skipping query.")
        return None

    # -------------------------------
    # 5. Build the SQL query
    # -------------------------------
    query = f"""
        SELECT 
            MakerId, 
            CoreSymbol, 
            TimeRecorded, 
            TimeSent, 
            TimeReceived, 
            Depth, 
            Side, 
            Price, 
            Size, 
            Provider, 
            IndicativeFlags, 
            QuoteFlags, 
            DisabledFlags, 
            ForwardPriceDelta, 
            id
        FROM Alp_Quotes PARTITION ({partition_name})
        FORCE INDEX (idx_time_recorded)
        WHERE 
            CoreSymbol = '{symbol}'
            AND TimeRecorded >= '{start_str}'
            AND TimeRecorded < '{end_str}';
    """
    
    ssh_host = '18.133.184.11'
    ssh_user = 'ubuntu'
    ssh_key_file = '/Users/jackhan/Desktop/Alpfin/OneZero_Data.pem'
    db_host = '127.0.0.1'
    db_port = 3306
    db_user = 'Ruize'
    db_password = 'Ma5hedPotato567='
    db_name = 'Alp_CPT_Data'
    
    columns = [
        "MakerId",
        "CoreSymbol",
        "TimeRecorded",
        "TimeSent",
        "TimeReceived",
        "Depth",
        "Side",
        "Price",
        "Size",
        "Provider",
        "IndicativeFlags",
        "QuoteFlags",
        "DisabledFlags",
        "ForwardPriceDelta",
        "id"
    ]
    
    try:
        with SSHTunnelForwarder(
            (ssh_host, 22),
            ssh_username=ssh_user,
            ssh_pkey=ssh_key_file,
            remote_bind_address=(db_host, db_port),
            allow_agent=False,
            host_pkey_directories=[]
        ) as tunnel:
            
            connection = pymysql.connect(
                host='127.0.0.1',
                port=tunnel.local_bind_port,
                user=db_user,
                password=db_password,
                database=db_name,
                connect_timeout=10
            )
            
            try:
                cursor = connection.cursor()
                start_time = time.time()
                
                print("Start Query: ", query)
                cursor.execute(query)
                print("Query Success in seconds: ", time.time() - start_time)
                rows = cursor.fetchall()
                print("Fetch Success")
                
                duration = time.time() - start_time
                print(f"[{symbol} | {date_str}] Fetched {len(rows)} rows in {duration:.2f} secs.")
                
                df = pd.DataFrame(rows, columns=columns)
                df.to_pickle(output_pickle)
                
                return df
            
            finally:
                cursor.close()
                connection.close()
                
    except Exception as e:
        print(f"ERROR for {symbol} on {date_str}: {str(e)}")
        return None
    

# # Example: get data for 2025-01-08, symbol 'XAU/USD'
# requested_date = pd.Timestamp("2025-01-08")
# df_result = get_quote_data(requested_date, "XAU/USD")

# if df_result is not None:
#     print(f"Fetched DataFrame with {len(df_result)} rows.")
# else:
#     print("No data returned or an error occurred.")

In [ ]:
symbols = [
    "AUD/CAD",
    "AUD/JPY",
    "AUD/USD",
    "BTCUSD",
    "CAD/JPY",
    "ETHUSD",
    "EUR/GBP",
    "EUR/NZD",
    "EUR/SEK",
    "EUR/USD",
    "EUR/ZAR",
    "GBP/JPY",
    "GBP/USD",
    "NASUSD",
    "NZD/USD",
    "U30USD",
    "USD/CAD",
    "USD/CHF",
    "USD/JPY",
    "USOUSD",
    "XAG/USD",
    "XAU/USD",
    "XNG/USD"
]

# 2. Create a date range for all *business* days (weekdays) in January 2025
#    (If you need *all* days including weekends, use freq="D" instead of "B".)
date_range = pd.date_range(start="2024-11-01", end="2025-01-23", freq="D")

# 3. For each date and symbol, call get_quote_data.
#    Note: This assumes you have already defined the get_quote_data() function 
#    in the same script or imported it from elsewhere.

for current_date in date_range:
    for symbol in symbols:
        print(f"Fetching data for {symbol} on {current_date.date()} ...")
        df = get_quote_data(current_date, symbol)
        # df is also saved automatically (via the get_quote_data() function)
        # to Data/<symbol>_<YYYY-MM-DD>.pkl
        if df is not None:
            print(f"  -> Returned {len(df)} rows.")
        else:
            print("  -> No data or error.")

Fetching data for AUD/CAD on 2024-11-01 ...
File Data/AUDCAD_2024-11-01.pkl already exists. Skipping query.
  -> No data or error.
Fetching data for AUD/JPY on 2024-11-01 ...
File Data/AUDJPY_2024-11-01.pkl already exists. Skipping query.
  -> No data or error.
Fetching data for AUD/USD on 2024-11-01 ...
File Data/AUDUSD_2024-11-01.pkl already exists. Skipping query.
  -> No data or error.
Fetching data for BTCUSD on 2024-11-01 ...
File Data/BTCUSD_2024-11-01.pkl already exists. Skipping query.
  -> No data or error.
Fetching data for CAD/JPY on 2024-11-01 ...
File Data/CADJPY_2024-11-01.pkl already exists. Skipping query.
  -> No data or error.
Fetching data for ETHUSD on 2024-11-01 ...
File Data/ETHUSD_2024-11-01.pkl already exists. Skipping query.
  -> No data or error.
Fetching data for EUR/GBP on 2024-11-01 ...
File Data/EURGBP_2024-11-01.pkl already exists. Skipping query.
  -> No data or error.
Fetching data for EUR/NZD on 2024-11-01 ...
File Data/EURNZD_2024-11-01.pkl already e

2025-01-29 02:33:03,255| ERROR   | Could not connect to gateway 18.133.184.11:22 : Unable to connect to 18.133.184.11: [Errno 60] Operation timed out


ERROR for NASUSD on 2024-11-08: Could not establish session to SSH gateway
  -> No data or error.
Fetching data for NZD/USD on 2024-11-08 ...


2025-01-29 02:34:18,352| ERROR   | Could not connect to gateway 18.133.184.11:22 : Unable to connect to 18.133.184.11: [Errno 60] Operation timed out


ERROR for NZD/USD on 2024-11-08: Could not establish session to SSH gateway
  -> No data or error.
Fetching data for U30USD on 2024-11-08 ...


...

ERROR for NZD/USD on 2024-12-10: Could not establish session to SSH gateway
  -> No data or error.
Fetching data for U30USD on 2024-12-10 ...


2025-01-29 17:01:40,412| ERROR   | Could not connect to gateway 18.133.184.11:22 : Unable to connect to 18.133.184.11: [Errno 60] Operation timed out


ERROR for U30USD on 2024-12-10: Could not establish session to SSH gateway
  -> No data or error.
Fetching data for USD/CAD on 2024-12-10 ...


2025-01-29 17:02:55,491| ERROR   | Could not connect to gateway 18.133.184.11:22 : Unable to connect to 18.133.184.11: [Errno 60] Operation timed out


ERROR for USD/CAD on 2024-12-10: Could not establish session to SSH gateway
  -> No data or error.
Fetching data for USD/CHF on 2024-12-10 ...
